In [ ]:


pip install bert-for-tf2

Collecting bert-for-tf2
  Using cached bert-for-tf2-0.14.4.tar.gz (40 kB)
Collecting py-params>=0.9.6
  Using cached py-params-0.9.7.tar.gz (6.8 kB)
Collecting params-flow>=0.8.0
  Using cached params-flow-0.8.2.tar.gz (22 kB)
Requirement already satisfied: numpy in /home/ubuntu/anaconda3/lib/python3.7/site-packages (from params-flow>=0.8.0->bert-for-tf2) (1.19.0)
Requirement already satisfied: tqdm in /home/ubuntu/anaconda3/lib/python3.7/site-packages (from params-flow>=0.8.0->bert-for-tf2) (4.36.1)
Building wheels for collected packages: bert-for-tf2, py-params, params-flow
  Building wheel for bert-for-tf2 (setup.py) ... done
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.4-py3-none-any.whl size=30114 sha256=67ab5905d5b428c108cf70865de2f3a6b45af286a8774431abf279ea20642731
  Stored in directory: /home/ubuntu/.cache/pip/wheels/6c/c9/9c/363182ea34a736dae336eeaf0dd4a7eec3c6a5afe32373e1fe
  Building wheel for py-params (setup.py) ... done
  Created wheel for py-params: filename=py_params-0.9.7-py3-none-any.whl size=7302 sha256=c9c62088cf52f05b8ea9af95884f6666eded71e66de724543940439c8421291e
  Stored in directory: /home/ubuntu/.cache/pip/wheels/47/3d/2d/bbffcfd6b9f4b8b5cbf07e7520ac2676192fe9431240c13ee8
  Building wheel for params-flow (setup.py) ... done
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19473 sha256=894b39438ee730df6e79913edef64f071a3b1300157468d9f53ca7c9f6fc9275
  Stored in directory: /home/ubuntu/.cache/pip/wheels/0e/fc/d2/a44fff33af0f233d7def6e7de413006d57c10e10ad736fe8f5
Successfully built bert-for-tf2 py-params params-flow
Installing collected packages: py-params, params-flow, bert-for-tf2
Successfully installed bert-for-tf2-0.14.4 params-flow-0.8.2 py-params-0.9.7
Note: you may need to restart the kernel to use updated packages.

from bert import BertModelLayer

l_bert = BertModelLayer(**BertModelLayer.Params(
  vocab_size               = 16000,        # embedding params
  use_token_type           = True,
  use_position_embeddings  = True,
  token_type_vocab_size    = 2,

  num_layers               = 12,           # transformer encoder params
  hidden_size              = 768,
  hidden_dropout           = 0.1,
  intermediate_size        = 4*768,
  intermediate_activation  = "gelu",

  adapter_size             = None,         # see arXiv:1902.00751 (adapter-BERT)

  shared_layer             = False,        # True for ALBERT (arXiv:1909.11942)
  embedding_size           = None        # None for BERT, wordpiece embedding size for ALBERT

  #name                     = "bert"        # any other Keras layer params
))

import bert

model_dir = "/home/ubuntu/anaconda3/lib/python3.7/site-packages/uncased_L-24_H-1024_A-16/"

bert_params = bert.params_from_pretrained_ckpt(model_dir)
l_bert = bert.BertModelLayer.from_params(bert_params, name="bert")

from tensorflow import keras

max_seq_len = 128
l_input_ids      = keras.layers.Input(shape=(max_seq_len,), dtype='int32')
l_token_type_ids = keras.layers.Input(shape=(max_seq_len,), dtype='int32')

# using the default token_type/segment id 0
output = l_bert(l_input_ids)                              # output: [batch_size, max_seq_len, hidden_size]
model = keras.Model(inputs=l_input_ids, outputs=output)
model.build(input_shape=(None, max_seq_len))

import os

bert_ckpt_file   = os.path.join(model_dir, "bert_model.ckpt")
bert.load_stock_weights(l_bert, bert_ckpt_file)

Done loading 388 BERT weights from: /home/ubuntu/anaconda3/lib/python3.7/site-packages/uncased_L-24_H-1024_A-16/bert_model.ckpt into <bert.model.BertModelLayer object at 0x7fce54768910> (prefix:bert). Count of weights not found in the checkpoint was: [0]. Count of weights with mismatched shape: [0]
Unused weights from checkpoint: 
	bert/embeddings/token_type_embeddings
	bert/pooler/dense/bias
	bert/pooler/dense/kernel
	cls/predictions/output_bias
	cls/predictions/transform/LayerNorm/beta
	cls/predictions/transform/LayerNorm/gamma
	cls/predictions/transform/dense/bias
	cls/predictions/transform/dense/kernel
	cls/seq_relationship/output_bias
	cls/seq_relationship/output_weights

[]

import tensorflow_hub as hub
import tensorflow as tf
import bert
FullTokenizer = bert.bert_tokenization.FullTokenizer
from tensorflow.keras.models import Model       # Keras is the new high level API for TensorFlow
import math

max_seq_length = 128  # Your choice here.
input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                       name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                   name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                    name="segment_ids")
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=True)
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])

model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=[pooled_output, sequence_output])

def get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))


def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))


def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

import tensorflow as tf
from tensorflow.keras import layers
import bert
import tensorflow_hub as hub
import pandas as pd
from io import StringIO

BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

s = "This is a nice sentence."
stokens = tokenizer.tokenize(s)
stokens = ["[CLS]"] + stokens + ["[SEP]"]

input_ids = get_ids(stokens, tokenizer, max_seq_length)
input_masks = get_masks(stokens, max_seq_length)
input_segments = get_segments(stokens, max_seq_length)

pool_embs, all_embs = model.predict([[input_ids],[input_masks],[input_segments]])

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-42-76c994e50726> in <module>
      1 s = "This is a nice sentence."
      2 stokens = tokenizer.tokenize(s)
----> 3 stokens = ["[CLS]"] + stokens + ["[SEP]"]
      4 
      5 input_ids = get_ids(stokens, tokenizer, max_seq_length)

TypeError: can only concatenate list (not "generator") to list

model.predict([[input_ids],[input_masks],[input_segments]])

WARNING:tensorflow:Model was constructed with shape (None, 128) for input Tensor("input_word_ids:0", shape=(None, 128), dtype=int32), but it was called on an input with incompatible shape (None, 1, 128).

WARNING:tensorflow:Model was constructed with shape (None, 128) for input Tensor("input_word_ids:0", shape=(None, 128), dtype=int32), but it was called on an input with incompatible shape (None, 1, 128).

---------------------------------------------------------------------------
AssertionError                            Traceback (most recent call last)
<ipython-input-53-cad552a10e5d> in <module>
----> 1 model.predict([[input_ids],[input_masks],[input_segments]])

~/.local/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py in _method_wrapper(self, *args, **kwargs)
     86       raise ValueError('{} is not supported in multi-worker mode.'.format(
     87           method.__name__))
---> 88     return method(self, *args, **kwargs)
     89 
     90   return tf_decorator.make_decorator(

~/.local/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py in predict(self, x, batch_size, verbose, steps, callbacks, max_queue_size, workers, use_multiprocessing)
   1266           for step in data_handler.steps():
   1267             callbacks.on_predict_batch_begin(step)
-> 1268             tmp_batch_outputs = predict_function(iterator)
   1269             # Catch OutOfRangeError for Datasets of unknown size.
   1270             # This blocks until the batch has finished executing.

~/.local/lib/python3.7/site-packages/tensorflow/python/eager/def_function.py in __call__(self, *args, **kwds)
    578         xla_context.Exit()
    579     else:
--> 580       result = self._call(*args, **kwds)
    581 
    582     if tracing_count == self._get_tracing_count():

~/.local/lib/python3.7/site-packages/tensorflow/python/eager/def_function.py in _call(self, *args, **kwds)
    616       # In this case we have not created variables on the first call. So we can
    617       # run the first trace but we should fail if variables are created.
--> 618       results = self._stateful_fn(*args, **kwds)
    619       if self._created_variables:
    620         raise ValueError("Creating variables on a non-first call to a function"

~/.local/lib/python3.7/site-packages/tensorflow/python/eager/function.py in __call__(self, *args, **kwargs)
   2417     """Calls a graph function specialized to the inputs."""
   2418     with self._lock:
-> 2419       graph_function, args, kwargs = self._maybe_define_function(args, kwargs)
   2420     return graph_function._filtered_call(args, kwargs)  # pylint: disable=protected-access
   2421 

~/.local/lib/python3.7/site-packages/tensorflow/python/eager/function.py in _maybe_define_function(self, args, kwargs)
   2772           and self.input_signature is None
   2773           and call_context_key in self._function_cache.missed):
-> 2774         return self._define_function_with_shape_relaxation(args, kwargs)
   2775 
   2776       self._function_cache.missed.add(call_context_key)

~/.local/lib/python3.7/site-packages/tensorflow/python/eager/function.py in _define_function_with_shape_relaxation(self, args, kwargs)
   2704         relaxed_arg_shapes)
   2705     graph_function = self._create_graph_function(
-> 2706         args, kwargs, override_flat_arg_shapes=relaxed_arg_shapes)
   2707     self._function_cache.arg_relaxed[rank_only_cache_key] = graph_function
   2708 

~/.local/lib/python3.7/site-packages/tensorflow/python/eager/function.py in _create_graph_function(self, args, kwargs, override_flat_arg_shapes)
   2665             arg_names=arg_names,
   2666             override_flat_arg_shapes=override_flat_arg_shapes,
-> 2667             capture_by_value=self._capture_by_value),
   2668         self._function_attributes,
   2669         # Tell the ConcreteFunction to clean up its graph once it goes out of

~/.local/lib/python3.7/site-packages/tensorflow/python/framework/func_graph.py in func_graph_from_py_func(name, python_func, args, kwargs, signature, func_graph, autograph, autograph_options, add_control_dependencies, arg_names, op_return_value, collections, capture_by_value, override_flat_arg_shapes)
    979         _, original_func = tf_decorator.unwrap(python_func)
    980 
--> 981       func_outputs = python_func(*func_args, **func_kwargs)
    982 
    983       # invariant: `func_outputs` contains only Tensors, CompositeTensors,

~/.local/lib/python3.7/site-packages/tensorflow/python/eager/def_function.py in wrapped_fn(*args, **kwds)
    439         # __wrapped__ allows AutoGraph to swap in a converted function. We give
    440         # the function a weak reference to itself to avoid a reference cycle.
--> 441         return weak_wrapped_fn().__wrapped__(*args, **kwds)
    442     weak_wrapped_fn = weakref.ref(wrapped_fn)
    443 

~/.local/lib/python3.7/site-packages/tensorflow/python/framework/func_graph.py in wrapper(*args, **kwargs)
    966           except Exception as e:  # pylint:disable=broad-except
    967             if hasattr(e, "ag_error_metadata"):
--> 968               raise e.ag_error_metadata.to_exception(e)
    969             else:
    970               raise

AssertionError: in user code:

    /home/ubuntu/.local/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1147 predict_function  *
        outputs = self.distribute_strategy.run(
    /home/ubuntu/.local/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/ubuntu/.local/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/ubuntu/.local/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/ubuntu/.local/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1122 predict_step  **
        return self(x, training=False)
    /home/ubuntu/.local/lib/python3.7/site-packages/tensorflow/python/keras/engine/base_layer.py:927 __call__
        outputs = call_fn(cast_inputs, *args, **kwargs)
    /home/ubuntu/.local/lib/python3.7/site-packages/tensorflow/python/keras/engine/network.py:719 call
        convert_kwargs_to_constants=base_layer_utils.call_context().saving)
    /home/ubuntu/.local/lib/python3.7/site-packages/tensorflow/python/keras/engine/network.py:899 _run_internal_graph
        assert str(id(x)) in tensor_dict, 'Could not compute output ' + str(x)

    AssertionError: Could not compute output Tensor("keras_layer/Identity:0", shape=(None, 768), dtype=float32)

s = "This is a nice sentence."
stokens = tokenizer.tokenize(s)

BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)
#input_ids = get_ids(stokens, tokenizer, max_seq_length)
token_ids = tokenizer.convert_tokens_to_ids(stokens)
input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
print(input_ids)

input_masks = get_masks(stokens, max_seq_length)
input_segments = get_segments(stokens, max_seq_length)

[2023, 2003, 1037, 3835, 6251, 1012, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

 

import tensorflow as tf
from tensorflow.keras import layers
import bert
import tensorflow_hub as hub
import pandas as pd
from io import StringIO

BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)
print(tokenizer.tokenize("don't be so judgmental"))
print(tokenizer.convert_tokens_to_ids(tokenizer.tokenize("dont be so judgmental")))

def tokenize_reviews(text_reviews):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_reviews))
#texts = pd.read_csv(StringIO("sam.txt"))
#f = open("sam.txt", "r")
#texts = f.read()
texts = 'My name is Saeedeh'
print(texts)
#tokenized_reviews = [tokenize_reviews(review) for review in texts]
print(tokenizer.tokenize(texts))
print('-----')
#print(tokenized_reviews)

['don', "'", 't', 'be', 'so', 'judgment', '##al']
[2123, 2102, 2022, 2061, 8689, 2389]
My name is Saeedeh
['my', 'name', 'is', 'sa', '##eed', '##eh']
-----

!wget https://github.com/lonePatient/bert-sentence-similarity-pytorch/archive/master.zip -O t3.zip

--2020-07-15 00:57:20--  https://github.com/lonePatient/bert-sentence-similarity-pytorch/archive/master.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/lonePatient/bert-sentence-similarity-pytorch/zip/master [following]
--2020-07-15 00:57:20--  https://codeload.github.com/lonePatient/bert-sentence-similarity-pytorch/zip/master
Resolving codeload.github.com (codeload.github.com)... 140.82.113.9
Connecting to codeload.github.com (codeload.github.com)|140.82.113.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘t3.zip’

t3.zip                  [ <=>                ]  40.92K  --.-KB/s    in 0.1s    

2020-07-15 00:57:21 (370 KB/s) - ‘t3.zip’ saved [41898]

ls

chinese_L-12_H-768_A-12.zip/  __init__.py  pytorch_pretrain/

!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip

--2020-07-15 01:55:33--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.14.208, 216.58.217.48, 172.217.14.240, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.14.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M  61.5MB/s    in 6.0s    

2020-07-15 01:55:39 (65.2 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]

ls

chinese_L-12_H-768_A-12.zip/  pytorch_pretrain/
__init__.py                   uncased_L-12_H-768_A-12.zip

pip install pytorch-pretrained-bert

Requirement already satisfied: pytorch-pretrained-bert in /home/ubuntu/anaconda3/lib/python3.7/site-packages (0.6.2)
Requirement already satisfied: tqdm in /home/ubuntu/anaconda3/lib/python3.7/site-packages (from pytorch-pretrained-bert) (4.36.1)
Requirement already satisfied: numpy in /home/ubuntu/anaconda3/lib/python3.7/site-packages (from pytorch-pretrained-bert) (1.19.0)
Requirement already satisfied: requests in /home/ubuntu/anaconda3/lib/python3.7/site-packages (from pytorch-pretrained-bert) (2.24.0)
Requirement already satisfied: regex in /home/ubuntu/anaconda3/lib/python3.7/site-packages (from pytorch-pretrained-bert) (2020.6.8)
Requirement already satisfied: boto3 in /home/ubuntu/anaconda3/lib/python3.7/site-packages (from pytorch-pretrained-bert) (1.14.17)
Requirement already satisfied: torch>=0.4.1 in /home/ubuntu/anaconda3/lib/python3.7/site-packages (from pytorch-pretrained-bert) (1.5.1)
Requirement already satisfied: idna<3,>=2.5 in /home/ubuntu/anaconda3/lib/python3.7/site-packages (from requests->pytorch-pretrained-bert) (2.10)
Requirement already satisfied: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in /home/ubuntu/anaconda3/lib/python3.7/site-packages (from requests->pytorch-pretrained-bert) (1.25.9)
Requirement already satisfied: certifi>=2017.4.17 in /home/ubuntu/anaconda3/lib/python3.7/site-packages (from requests->pytorch-pretrained-bert) (2020.6.20)
Requirement already satisfied: chardet<4,>=3.0.2 in /home/ubuntu/anaconda3/lib/python3.7/site-packages (from requests->pytorch-pretrained-bert) (3.0.4)
Requirement already satisfied: botocore<1.18.0,>=1.17.17 in /home/ubuntu/anaconda3/lib/python3.7/site-packages (from boto3->pytorch-pretrained-bert) (1.17.17)
Requirement already satisfied: s3transfer<0.4.0,>=0.3.0 in /home/ubuntu/anaconda3/lib/python3.7/site-packages (from boto3->pytorch-pretrained-bert) (0.3.3)
Requirement already satisfied: jmespath<1.0.0,>=0.7.1 in /home/ubuntu/anaconda3/lib/python3.7/site-packages (from boto3->pytorch-pretrained-bert) (0.10.0)
Requirement already satisfied: future in /home/ubuntu/anaconda3/lib/python3.7/site-packages (from torch>=0.4.1->pytorch-pretrained-bert) (0.17.1)
Requirement already satisfied: python-dateutil<3.0.0,>=2.1 in /home/ubuntu/anaconda3/lib/python3.7/site-packages (from botocore<1.18.0,>=1.17.17->boto3->pytorch-pretrained-bert) (2.8.0)
Requirement already satisfied: docutils<0.16,>=0.10 in /home/ubuntu/anaconda3/lib/python3.7/site-packages (from botocore<1.18.0,>=1.17.17->boto3->pytorch-pretrained-bert) (0.15.2)
Requirement already satisfied: six>=1.5 in /home/ubuntu/anaconda3/lib/python3.7/site-packages (from python-dateutil<3.0.0,>=2.1->botocore<1.18.0,>=1.17.17->boto3->pytorch-pretrained-bert) (1.15.0)
Note: you may need to restart the kernel to use updated packages.

!python convert_tf_checkpoint_to_pytorch.py

cp: cannot stat 'pybert/model/pretrain/chinese_L-12_H-768_A-12/bert_config.json': No such file or directory
Traceback (most recent call last):
  File "convert_tf_checkpoint_to_pytorch.py", line 11, in <module>
    config['pytorch_model_path'])
  File "/home/ubuntu/virt-env/lib/python3.6/site-packages/pytorch_pretrained_bert/convert_tf_checkpoint_to_pytorch.py", line 32, in convert_tf_checkpoint_to_pytorch
    config = BertConfig.from_json_file(bert_config_file)
  File "/home/ubuntu/virt-env/lib/python3.6/site-packages/pytorch_pretrained_bert/modeling.py", line 206, in from_json_file
    with open(json_file, "r", encoding='utf-8') as reader:
FileNotFoundError: [Errno 2] No such file or directory: 'pybert/model/pretrain/chinese_L-12_H-768_A-12/bert_config.json'

!wget https://github.com/Brokenwind/BertSimilarity/archive/master.zip -O t4.zip

--2020-07-15 02:05:55--  https://github.com/Brokenwind/BertSimilarity/archive/master.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/Brokenwind/BertSimilarity/zip/master [following]
--2020-07-15 02:05:55--  https://codeload.github.com/Brokenwind/BertSimilarity/zip/master
Resolving codeload.github.com (codeload.github.com)... 140.82.113.9
Connecting to codeload.github.com (codeload.github.com)|140.82.113.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘t4.zip’

t4.zip                  [    <=>             ]   2.81M  3.62MB/s    in 0.8s    

2020-07-15 02:05:57 (3.62 MB/s) - ‘t4.zip’ saved [2951864]

!unzip t4.zip

Archive:  t4.zip
fecd3f6cf27988389d15a97dea52fc27ca553736
   creating: BertSimilarity-master/
  inflating: BertSimilarity-master/.gitignore  
  inflating: BertSimilarity-master/README.md  
  inflating: BertSimilarity-master/__init__.py  
  inflating: BertSimilarity-master/args.py  
   creating: BertSimilarity-master/data/
   creating: BertSimilarity-master/data/dataset/
  inflating: BertSimilarity-master/data/dataset/dev.csv  
  inflating: BertSimilarity-master/data/dataset/train.csv  
   creating: BertSimilarity-master/logs/
  inflating: BertSimilarity-master/logs/loss.log  
  inflating: BertSimilarity-master/logs/train.log  
  inflating: BertSimilarity-master/modeling.py  
  inflating: BertSimilarity-master/optimization.py  
 extracting: BertSimilarity-master/requirements.txt  
   creating: BertSimilarity-master/shell/
  inflating: BertSimilarity-master/shell/extract_log.sh  
  inflating: BertSimilarity-master/similarity.py  
  inflating: BertSimilarity-master/start.sh  
  inflating: BertSimilarity-master/tokenization.py  
  inflating: BertSimilarity-master/visilize.py  
   creating: BertSimilarity-master/wiki/
   creating: BertSimilarity-master/wiki/diagram/
  inflating: BertSimilarity-master/wiki/diagram/bert_similarity.drawio  
   creating: BertSimilarity-master/wiki/pictures/
  inflating: BertSimilarity-master/wiki/pictures/bert_embedding.png  
  inflating: BertSimilarity-master/wiki/pictures/bert_similarity.png  
  inflating: BertSimilarity-master/wiki/pictures/similarity_bert_model.png  
  inflating: BertSimilarity-master/wiki/pictures/similarity_data_process.png  
 extracting: BertSimilarity-master/wiki/pictures/similarity_eualtion_1.png  
  inflating: BertSimilarity-master/wiki/pictures/similarity_eval.png  
  inflating: BertSimilarity-master/wiki/pictures/similarity_eval_1.png  
  inflating: BertSimilarity-master/wiki/pictures/similarity_infer.png  
  inflating: BertSimilarity-master/wiki/pictures/similarity_train.png  

!wget https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip t4.zip

--2020-07-15 03:44:29--  https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 216.58.217.48, 172.217.14.208, 172.217.14.240, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|216.58.217.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 381892918 (364M) [application/zip]
Saving to: ‘chinese_L-12_H-768_A-12.zip’

chinese_L-12_H-768_ 100%[===================>] 364.20M  25.1MB/s    in 14s     

2020-07-15 03:44:43 (26.1 MB/s) - ‘chinese_L-12_H-768_A-12.zip’ saved [381892918/381892918]

--2020-07-15 03:44:43--  http://t4.zip/
Resolving t4.zip (t4.zip)... failed: Name or service not known.
wget: unable to resolve host address ‘t4.zip’
FINISHED --2020-07-15 03:44:44--
Total wall clock time: 15s
Downloaded: 1 files, 364M in 14s (26.1 MB/s)

cp -r BertSimilarity-master/data/pretrained_model/directory/chinese_L-12_H-768_A-12  BertSimilarity-master/data/pretrained_model/chinese_L-12_H-768_A-12

cp -r anaconda3/lib/python3.7/site-packages/uncased_L-24_H-1024_A-16  virt-env/jupyterNB/BertSimilarity-master/data/pretrained_model/directory

!unzip chinese_L-12_H-768_A-12.zip

Archive:  chinese_L-12_H-768_A-12.zip
   creating: chinese_L-12_H-768_A-12/
  inflating: chinese_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: chinese_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: chinese_L-12_H-768_A-12/vocab.txt  
  inflating: chinese_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: chinese_L-12_H-768_A-12/bert_config.json  

cd

/home/ubuntu

!sed -i 's/from tensorflow.tools.compatibility import ast_editsf/import ast_edits/g'  /home/ubuntu/.local/lib/python3.7/site-packages/tensorflow/tools/compatibility/tf_upgrade_v2_main.py

!sed -i 's/tf.logging/tf.compat.v1.logging/g' /home/ubuntu/virt-env/jupyterNB/BertSimilarity-master/optimization.py

!sed -i 's/tf.app/tf.compat.v1.app/g' /home/ubuntu/virt-env/jupyterNB/BertSimilarity-master/tokenization.py

!sed -i 's/tf.app/tf.compat.v1.app/g' /home/ubuntu/virt-env/jupyterNB/BertSimilarity-master/similarity.py

!sed -i 's/tf.flags/tf.compat.v1.flags/g' /home/ubuntu/virt-env/jupyterNB/BertSimilarity-master/similarity.py

!sed -i 's/tf.train.Optimizer/tf.compat.v1.train.Optimizer/g' /home/ubuntu/virt-env/jupyterNB/BertSimilarity-master/similarity.py

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

import tensorflow as tf
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

from absl import logging

ls

directory/

import tensorflow.compat.v1 as tf
tf.enable_v2_behavior()

!pip install --upgrade tensorflow== 1.14.0

ERROR: Could not find a version that satisfies the requirement tensorflow== (from versions: 0.12.1, 1.0.0, 1.0.1, 1.1.0rc0, 1.1.0rc1, 1.1.0rc2, 1.1.0, 1.2.0rc0, 1.2.0rc1, 1.2.0rc2, 1.2.0, 1.2.1, 1.3.0rc0, 1.3.0rc1, 1.3.0rc2, 1.3.0, 1.4.0rc0, 1.4.0rc1, 1.4.0, 1.4.1, 1.5.0rc0, 1.5.0rc1, 1.5.0, 1.5.1, 1.6.0rc0, 1.6.0rc1, 1.6.0, 1.7.0rc0, 1.7.0rc1, 1.7.0, 1.7.1, 1.8.0rc0, 1.8.0rc1, 1.8.0, 1.9.0rc0, 1.9.0rc1, 1.9.0rc2, 1.9.0, 1.10.0rc0, 1.10.0rc1, 1.10.0, 1.10.1, 1.11.0rc0, 1.11.0rc1, 1.11.0rc2, 1.11.0, 1.12.0rc0, 1.12.0rc1, 1.12.0rc2, 1.12.0, 1.12.2, 1.12.3, 1.13.0rc0, 1.13.0rc1, 1.13.0rc2, 1.13.1, 1.13.2, 1.14.0rc0, 1.14.0rc1, 1.14.0, 1.15.0rc0, 1.15.0rc1, 1.15.0rc2, 1.15.0rc3, 1.15.0, 1.15.2, 1.15.3, 2.0.0a0, 2.0.0b0, 2.0.0b1, 2.0.0rc0, 2.0.0rc1, 2.0.0rc2, 2.0.0, 2.0.1, 2.0.2, 2.1.0rc0, 2.1.0rc1, 2.1.0rc2, 2.1.0, 2.1.1, 2.2.0rc0, 2.2.0rc1, 2.2.0rc2, 2.2.0rc3, 2.2.0rc4, 2.2.0, 2.3.0rc0, 2.3.0rc1)
ERROR: No matching distribution found for tensorflow==

cd uncased_L-24_H-1024_A-16/

/home/ubuntu/anaconda3/lib/python3.7/site-packages/uncased_L-24_H-1024_A-16

ls

bert_config.json                     bert_model.ckpt.index  vocab.txt
bert_model.ckpt.data-00000-of-00001  bert_model.ckpt.meta

cd chinese_L-12_H-768_A-12/

/home/ubuntu/virt-env/jupyterNB/BertSimilarity-master/data/pretrained_model/chinese_L-12_H-768_A-12

cp /home/ubuntu/anaconda3/lib/python3.7/site-packages/uncased_L-24_H-1024_A-16/bert_model.ckpt.meta /home/ubuntu/virt-env/jupyterNB/BertSimilarity-master/data/pretrained_model/chinese_L-12_H-768_A-12/

!pip install keras-layer-normalization

Collecting keras-layer-normalization
  Using cached keras-layer-normalization-0.14.0.tar.gz (4.3 kB)
Requirement already satisfied: numpy in /home/ubuntu/virt-env/lib/python3.6/site-packages (from keras-layer-normalization) (1.19.0)
Requirement already satisfied: Keras in /home/ubuntu/virt-env/lib/python3.6/site-packages (from keras-layer-normalization) (2.3.1)
Requirement already satisfied: scipy>=0.14 in /home/ubuntu/virt-env/lib/python3.6/site-packages (from Keras->keras-layer-normalization) (1.4.1)
Requirement already satisfied: six>=1.9.0 in /home/ubuntu/virt-env/lib/python3.6/site-packages (from Keras->keras-layer-normalization) (1.15.0)
Requirement already satisfied: keras-preprocessing>=1.0.5 in /home/ubuntu/virt-env/lib/python3.6/site-packages (from Keras->keras-layer-normalization) (1.1.2)
Requirement already satisfied: h5py in /home/ubuntu/virt-env/lib/python3.6/site-packages (from Keras->keras-layer-normalization) (2.10.0)
Requirement already satisfied: pyyaml in /home/ubuntu/virt-env/lib/python3.6/site-packages (from Keras->keras-layer-normalization) (5.3.1)
Requirement already satisfied: keras-applications>=1.0.6 in /home/ubuntu/virt-env/lib/python3.6/site-packages (from Keras->keras-layer-normalization) (1.0.8)
Building wheels for collected packages: keras-layer-normalization
  Building wheel for keras-layer-normalization (setup.py) ... done
  Created wheel for keras-layer-normalization: filename=keras_layer_normalization-0.14.0-py3-none-any.whl size=5267 sha256=83ec855c2412e117cebee6ab834a5f277b4c728f3d52a79f3f7358451e6d8650
  Stored in directory: /home/ubuntu/.cache/pip/wheels/60/1a/38/858ffe627cf272dc54d9863d6c5ec993f00fd28d33f7f169f8
Successfully built keras-layer-normalization
Installing collected packages: keras-layer-normalization
Successfully installed keras-layer-normalization-0.14.0

pip install sentence-transformers

Collecting sentence-transformers
  Using cached sentence-transformers-0.3.0.tar.gz (61 kB)
Collecting transformers>=3.0.2
  Using cached transformers-3.0.2-py3-none-any.whl (769 kB)
Requirement already satisfied: tqdm in /home/ubuntu/anaconda3/lib/python3.7/site-packages (from sentence-transformers) (4.36.1)
Requirement already satisfied: torch>=1.0.1 in /home/ubuntu/anaconda3/lib/python3.7/site-packages (from sentence-transformers) (1.5.1)
Requirement already satisfied: numpy in /home/ubuntu/anaconda3/lib/python3.7/site-packages (from sentence-transformers) (1.19.0)
Requirement already satisfied: scikit-learn in /home/ubuntu/anaconda3/lib/python3.7/site-packages (from sentence-transformers) (0.21.3)
Requirement already satisfied: scipy in /home/ubuntu/anaconda3/lib/python3.7/site-packages (from sentence-transformers) (1.4.1)
Requirement already satisfied: nltk in /home/ubuntu/anaconda3/lib/python3.7/site-packages (from sentence-transformers) (3.4.5)
Collecting tokenizers==0.8.1.rc1
  Downloading tokenizers-0.8.1rc1-cp37-cp37m-manylinux1_x86_64.whl (3.0 MB)
     |████████████████████████████████| 3.0 MB 1.5 MB/s eta 0:00:01
Requirement already satisfied: sentencepiece!=0.1.92 in /home/ubuntu/anaconda3/lib/python3.7/site-packages (from transformers>=3.0.2->sentence-transformers) (0.1.91)
Collecting sacremoses
  Using cached sacremoses-0.0.43.tar.gz (883 kB)
Requirement already satisfied: regex!=2019.12.17 in /home/ubuntu/anaconda3/lib/python3.7/site-packages (from transformers>=3.0.2->sentence-transformers) (2020.6.8)
Requirement already satisfied: filelock in /home/ubuntu/anaconda3/lib/python3.7/site-packages (from transformers>=3.0.2->sentence-transformers) (3.0.12)
Requirement already satisfied: packaging in /home/ubuntu/anaconda3/lib/python3.7/site-packages (from transformers>=3.0.2->sentence-transformers) (19.2)
Requirement already satisfied: requests in /home/ubuntu/anaconda3/lib/python3.7/site-packages (from transformers>=3.0.2->sentence-transformers) (2.24.0)
Requirement already satisfied: future in /home/ubuntu/anaconda3/lib/python3.7/site-packages (from torch>=1.0.1->sentence-transformers) (0.17.1)
Requirement already satisfied: joblib>=0.11 in /home/ubuntu/anaconda3/lib/python3.7/site-packages (from scikit-learn->sentence-transformers) (0.13.2)
Requirement already satisfied: six in /home/ubuntu/anaconda3/lib/python3.7/site-packages (from nltk->sentence-transformers) (1.15.0)
Requirement already satisfied: click in /home/ubuntu/anaconda3/lib/python3.7/site-packages (from sacremoses->transformers>=3.0.2->sentence-transformers) (7.0)
Requirement already satisfied: pyparsing>=2.0.2 in /home/ubuntu/anaconda3/lib/python3.7/site-packages (from packaging->transformers>=3.0.2->sentence-transformers) (2.4.2)
Requirement already satisfied: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in /home/ubuntu/anaconda3/lib/python3.7/site-packages (from requests->transformers>=3.0.2->sentence-transformers) (1.25.9)
Requirement already satisfied: chardet<4,>=3.0.2 in /home/ubuntu/anaconda3/lib/python3.7/site-packages (from requests->transformers>=3.0.2->sentence-transformers) (3.0.4)
Requirement already satisfied: certifi>=2017.4.17 in /home/ubuntu/anaconda3/lib/python3.7/site-packages (from requests->transformers>=3.0.2->sentence-transformers) (2020.6.20)
Requirement already satisfied: idna<3,>=2.5 in /home/ubuntu/anaconda3/lib/python3.7/site-packages (from requests->transformers>=3.0.2->sentence-transformers) (2.10)
Building wheels for collected packages: sentence-transformers, sacremoses
  Building wheel for sentence-transformers (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.0-py3-none-any.whl size=86752 sha256=20c988cb39036f9955bcf4416279984d3aee44d76cbfbae4d0b4e9b52440c061
  Stored in directory: /home/ubuntu/.cache/pip/wheels/3e/15/94/49bc84289d2c77b5059bca513f840c6006d4e2cc7f10275d49
  Building wheel for sacremoses (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.43-py3-none-any.whl size=893259 sha256=dddb8e0f33bdbfc131d475a9d86bba1b6449c6d7248a262dc25488f92a61b642
  Stored in directory: /home/ubuntu/.cache/pip/wheels/69/09/d1/bf058f7d6fa0ecba2ce7c66be3b8d012beb4bf61a6e0c101c0
Successfully built sentence-transformers sacremoses
Installing collected packages: tokenizers, sacremoses, transformers, sentence-transformers
Successfully installed sacremoses-0.0.43 sentence-transformers-0.3.0 tokenizers-0.8.1rc1 transformers-3.0.2
Note: you may need to restart the kernel to use updated packages.

from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

100%|██████████| 405M/405M [00:36<00:00, 11.0MB/s] 

sentences = ['I am Saeedeh', 
             'My name is Saeedeh']

sentence_embeddings = model.encode(sentences)

print('Sample BERT embedding vector - length', len(sentence_embeddings[0]))

Sample BERT embedding vector - length 768

print('Sample BERT embedding vector - note includes negative values', sentence_embeddings[0])

Sample BERT embedding vector - note includes negative values [-2.35771760e-01  1.36883810e-01  1.84800053e+00 -1.25811979e-01
  2.68786281e-01  7.82029480e-02 -8.44726712e-02  6.24911308e-01
  1.99581146e-01 -1.58082455e-01 -6.44715488e-01  2.47140273e-01
  6.30017698e-01  6.35218561e-01  1.24508750e+00  3.77005160e-01
 -9.34593499e-01 -9.78802815e-02  6.96627975e-01 -4.92876232e-01
 -2.46506572e-01  4.76267189e-01  4.73605506e-02 -1.14886689e+00
 -9.21256915e-02 -7.96880603e-01  2.80487388e-01 -1.19131911e+00
 -2.57875174e-01 -1.37707144e-01  9.88038108e-02 -8.92846286e-01
  5.10215521e-01  4.17575659e-03 -3.06482557e-02  5.62663257e-01
 -4.67827231e-01  6.27215981e-01 -9.92233157e-02 -8.42029929e-01
  9.90488172e-01 -8.98426846e-02  9.36129987e-01  3.39323133e-01
 -6.28417909e-01 -5.21942794e-01  3.51907820e-01  8.46511405e-03
 -2.09809393e-01 -1.38582408e+00 -6.02593243e-01 -5.74002862e-01
  4.00717825e-01  2.68556505e-01 -6.64229468e-02  1.98734626e-01
 -3.66724521e-01 -8.66532505e-01 -4.69976664e-01  3.70891094e-02
  1.22992432e+00 -6.84941590e-01  8.52247059e-01  7.63499439e-01
 -8.56423844e-03  9.48563695e-01  3.33434165e-01 -5.66025436e-01
 -5.55821538e-01  7.53902912e-01  8.26268375e-01 -3.24228317e-01
 -3.69458526e-01  1.46105990e-01 -1.56266022e+00 -6.04295313e-01
  4.81327802e-01  4.23838824e-01  6.26195431e-01  8.51110041e-01
  7.16613233e-01 -1.89078480e-01  9.95296478e-01  5.87706447e-01
 -2.87997246e-01  9.22737122e-01 -2.09404498e-01  2.39145383e-01
 -1.58695948e+00  1.03202033e+00 -4.34548885e-01  5.28648019e-01
 -6.27690926e-02  2.23811358e-01  1.12788759e-01 -4.74804372e-01
  3.30129832e-01 -2.82596588e-01  4.14399296e-01 -5.17566323e-01
 -8.98373306e-01 -6.46069646e-01 -7.21665978e-01 -2.16665715e-01
  2.80940644e-02  5.11306226e-01 -4.29399580e-01 -5.69297493e-01
 -2.89131910e-01  5.25605917e-01  4.44211066e-01  1.89951703e-01
  6.44592404e-01 -2.20540255e-01 -1.01734780e-01 -1.72880054e-01
 -1.03372765e+00  1.01672923e-02 -1.53856114e-01  7.97433853e-01
  2.13383913e-01  6.50889456e-01 -3.54199350e-01 -5.33594728e-01
  6.19567096e-01 -4.81191605e-01  2.12624937e-01  2.14268163e-01
 -1.36364305e+00  4.89097923e-01  1.68143049e-01  1.43756956e-01
  3.61490965e-01  2.17651591e-01  1.41870901e-01 -3.97529066e-01
  2.67715573e-01 -3.67485911e-01  2.82720506e-01 -2.54263841e-02
 -9.88909721e-01  6.64506555e-01 -1.01875770e+00  3.06741685e-01
  2.74515837e-01 -3.40668671e-02 -1.18389465e-01  7.67451882e-01
 -5.75250328e-01  3.14521730e-01 -6.58088028e-01  3.44608843e-01
 -8.02685097e-02 -6.07824743e-01 -9.99060035e-01 -1.19055517e-01
 -1.11100709e+00  1.25586820e+00  8.95919278e-02  4.03460890e-01
 -1.88161612e-01  3.35448682e-01  5.57076573e-01 -2.73535222e-01
  2.52279192e-01 -8.74084309e-02  4.84659135e-01  2.39022821e-01
  1.04657292e+00 -1.11000173e-01  6.05911374e-01 -1.39636493e+00
 -1.22732170e-01  6.94532037e-01 -2.41219308e-02  4.83386606e-01
  3.23005050e-01  1.12779868e+00  7.66790390e-01 -3.47708672e-01
  9.52946767e-02  6.16207160e-02  7.76217878e-01 -5.77616632e-01
 -4.86781150e-01 -3.87087196e-01 -1.78863987e-01 -2.54674137e-01
  5.70562303e-01  5.95622420e-01 -2.97971755e-01 -5.82350194e-01
  2.33025774e-01 -1.25412691e+00  4.05036472e-02  3.98380697e-01
  3.85626778e-02  6.64132893e-01 -2.57497281e-01  1.55389428e-01
  1.20977485e+00 -3.60999167e-01  3.82894009e-01 -6.90741479e-01
  6.42949462e-01  1.17147195e+00 -2.68931955e-01  8.86982799e-01
  5.15734076e-01 -1.42212167e-01 -7.63814509e-01 -1.73200175e-01
  4.47638422e-01 -6.06674962e-02 -1.18338980e-01 -9.37052727e-01
  6.70516610e-01 -8.55556786e-01 -8.19137514e-01  2.14029685e-01
  5.59922695e-01  2.28806958e-01 -1.23456076e-01  4.06785339e-01
 -8.21892619e-01  5.08206990e-03  5.78719020e-01 -5.39367318e-01
 -1.59901261e-01 -8.02845597e-01  8.17502975e-01 -2.70362705e-01
  3.84975165e-01  5.96642017e-01  7.28524148e-01  5.04931748e-01
 -7.34081641e-02 -2.78917432e-01  1.54153943e-01  7.98756301e-01
  3.73495162e-01  8.51278007e-01  3.42795908e-01  4.22297716e-01
 -1.20486283e+00 -4.98730630e-01  9.43667114e-01 -4.00149733e-01
 -6.79179192e-01  8.84584248e-01 -2.78653890e-01 -9.44831192e-01
 -1.64600283e-01 -4.79329586e-01 -2.75208533e-01  1.78693384e-02
 -6.69798315e-01  1.95467710e-01  2.27543801e-01 -4.00851309e-01
 -4.91517305e-01  2.12963417e-01 -4.39501435e-01  1.81065992e-01
 -1.35675415e-01  4.06135879e-02  8.36359486e-02  2.37582535e-01
 -6.17810152e-02  7.19094574e-01  1.12460651e-01  1.19875729e-01
  4.02544349e-01 -4.44351524e-01  3.59473765e-01  7.09158108e-02
  1.00319374e+00  2.96303600e-01 -5.96531928e-01  9.18196961e-02
  2.75403764e-02 -1.05929069e-01 -1.90924907e+00  1.23849399e-01
  5.25901020e-01 -6.91190541e-01  6.99465394e-01 -1.01486254e+00
 -9.59778607e-01  4.23699021e-02 -4.05061662e-01 -5.24119556e-01
 -1.27926663e-01 -1.07638609e+00  8.53149414e-01 -1.17891669e+00
  1.50053814e-01  1.31978467e-01 -1.35561013e+00  3.27988118e-01
 -8.87534261e-01  1.84728727e-01 -5.57839453e-01 -9.82323214e-02
  7.96547472e-01 -5.24306357e-01 -1.52791515e-01  1.23831525e-01
 -2.41386557e+00 -3.96340400e-01 -1.24618185e+00  4.89665382e-02
  5.33265054e-01 -1.27877966e-01 -4.57836151e-01  5.84077202e-02
 -5.12748718e-01  5.51829040e-01  8.24887812e-01 -5.35996735e-01
 -4.27133173e-01 -3.37368578e-01  5.81975356e-02  1.89737216e-01
  3.74910027e-01 -1.16088353e-01 -6.98635459e-01  7.80031264e-01
 -2.08737895e-01 -3.12656492e-01  1.89829558e-01 -1.68690547e-01
  4.51322556e-01 -4.49245833e-02 -1.20378494e-01  6.56602442e-01
 -1.08209830e-02 -5.92614174e-01  1.05618155e+00  6.32579671e-03
 -2.54822642e-01 -4.11843002e-01  1.81877628e-01 -4.26571578e-01
  1.23699057e+00 -7.74604678e-01 -1.58561185e-01 -1.85238421e+00
  5.50647438e-01 -6.92240417e-01 -1.57026082e-01  4.78132963e-02
 -8.57857287e-01 -9.59437713e-02 -9.31144595e-01  2.80568093e-01
  2.85091460e-01 -2.61936992e-01 -4.43111323e-02  2.40339115e-01
  4.40722667e-02  5.67983575e-02 -4.57595557e-01  2.68868208e-01
 -2.34205052e-01 -2.41766319e-01  7.44834781e-01  1.79029539e-01
  1.41510737e+00  2.03923121e-01  5.06435990e-01 -8.92461121e-01
  3.05611342e-01  1.46074548e-01 -8.87752175e-02  3.04972470e-01
  1.74730808e-01 -6.39564872e-01 -1.01399846e-01 -6.12297118e-01
  2.59872586e-01 -1.37022793e+00  2.00632468e-01 -1.27148539e-01
  4.95399505e-01  2.95342982e-01 -4.87873226e-01 -7.78595090e-01
  6.06294990e-01 -4.71348375e-01  1.36371315e+00  9.65137929e-02
  1.59652516e-01 -5.94780333e-02  9.89113078e-02 -1.99532473e+00
  1.13487840e+00  7.55365312e-01 -2.83466309e-01  9.07832012e-02
  1.39049187e-01  6.03836179e-01  2.35528305e-01 -3.93693119e-01
  1.95108518e-01 -6.81405842e-01  5.20375311e-01 -5.70937216e-01
 -8.37077260e-01 -1.96805224e-01  2.83478498e-02  4.19492006e-01
 -3.43815647e-02  3.47747803e-01 -7.94583738e-01  6.10339582e-01
  3.80731523e-01 -1.13620555e+00  9.16510284e-01  2.02389851e-01
 -3.26076031e-01 -4.62780952e-01 -3.79143953e-01  2.40801468e-01
 -1.29712194e-01  6.09588504e-01 -3.71797591e-01  5.21587487e-03
 -1.67041689e-01 -3.86464387e-01 -6.46426678e-01  5.17688632e-01
 -7.22794175e-01  8.21035028e-01  6.46192491e-01 -1.05381310e-01
 -8.25911015e-02  4.29981709e-01 -1.22355856e-01 -3.69445682e-02
 -2.49350205e-01 -1.18937612e-01 -5.68530679e-01  4.94870823e-03
 -1.96922440e-02 -2.36430410e-02 -8.49393427e-01 -3.63766760e-01
  2.62200862e-01 -3.55726391e-01  4.65157539e-01  6.36879921e-01
  4.54771519e-01 -5.52051365e-02  9.76393640e-01 -1.25422105e-01
 -1.00169353e-01 -1.67926937e-01  3.22648734e-01 -2.20812201e-01
  1.23226285e-01 -3.24389905e-01 -5.99075913e-01 -7.23989457e-02
  4.18644011e-01 -4.69175577e-02 -1.14070058e+00  7.22452998e-01
 -2.41257295e-01 -6.30725622e-02 -2.83877015e-01  1.40128285e-01
  3.01281512e-01  4.88613337e-01  3.34253520e-01  8.57980132e-01
 -1.16975284e+00  3.18973988e-01  1.52653143e-01 -4.29636240e-02
 -9.85699371e-02  2.79123157e-01 -4.03498076e-02  3.30436051e-01
 -5.43219864e-01  9.36083853e-01  9.13697064e-01 -2.31673315e-01
  7.73057520e-01  2.26613298e-01 -6.68351293e-01 -7.55450428e-01
 -7.63749897e-01 -7.74165034e-01  9.07111093e-02  4.62972403e-01
 -4.94452000e-01  1.01996232e-02  3.81498635e-01  9.86320972e-02
 -2.47426301e-01  1.04581572e-01 -3.47994894e-01 -6.72631323e-01
  7.30797172e-01 -4.14536484e-02 -2.09981203e-01  6.88084245e-01
  2.37120059e-03  2.67532676e-01 -8.00127447e-01  1.22851539e+00
 -2.95068800e-01 -1.91773921e-01 -7.18454540e-01  1.77685052e-01
  2.07581639e-01  2.27213547e-01  2.44560286e-01 -2.90846288e-01
  4.20325220e-01  4.90175396e-01 -6.03981137e-01 -5.98957717e-01
 -1.41396254e-01  1.96566686e-01  8.98098528e-01 -2.81441838e-01
  6.11747265e-01 -6.03460193e-01  3.20750266e-01 -1.80126593e-01
 -1.79337278e-01  6.18677616e-01  1.79846987e-01  4.18161124e-01
 -8.50918710e-01 -3.76187742e-01  6.47974536e-02 -2.42263940e-03
  5.88478446e-01  8.45746472e-02 -4.91939843e-01 -8.40539277e-01
 -1.00212741e+00  3.34172904e-01 -3.52506429e-01 -5.77539444e-01
  4.14205283e-01 -1.27806640e+00 -6.99783981e-01 -4.29476768e-01
 -1.80982798e-02 -7.50556350e-01 -5.13301730e-01 -4.67274398e-01
  2.58272171e-01  9.99046862e-01 -2.56341070e-01 -4.03530091e-01
  1.28967911e-01 -2.71007210e-01 -3.37285012e-01  1.81873590e-01
 -3.07981670e-01  4.58019167e-01 -2.94220120e-01 -5.62904850e-02
  6.52068019e-01  2.39781335e-01 -3.99744987e-01 -3.69540125e-01
 -3.94526720e-01  7.35176742e-01 -4.72554982e-01 -3.00632328e-01
  1.20851922e+00  1.38577178e-01 -1.31115985e+00 -5.82796097e-01
  2.90958107e-01  1.06938624e+00  1.87813509e-02 -8.33555698e-01
 -8.86886299e-01  1.00683264e-01  9.67310548e-01 -8.10835958e-01
 -1.61643356e-01  6.92167640e-01  9.38608289e-01  4.01241779e-01
  9.86082375e-01 -3.28970224e-01  4.52778399e-01 -2.15466812e-01
 -3.46701294e-01  1.32433355e+00  9.73718532e-04  9.17837977e-01
 -1.29850417e-01  5.80981791e-01 -8.88177872e-01  9.96030152e-01
  2.36769184e-01  7.53487587e-01 -5.42549372e-01 -1.57172844e-01
  5.77571690e-01  5.17533600e-01 -1.11196823e-01 -3.60283822e-01
 -3.73153836e-01 -1.69427827e-01 -3.90640587e-01 -9.07781601e-01
 -4.43424553e-01  2.24100742e-02  9.44216907e-01  4.14521545e-01
  3.65376920e-01 -1.29891559e-01 -1.79625034e-01 -1.97979912e-01
 -2.99489588e-01 -1.30237132e-01 -1.90523386e-01 -3.16123277e-01
 -4.21428025e-01  4.22265321e-01  5.77929914e-01  5.26329875e-01
  5.81918597e-01 -1.22250222e-01  5.17626941e-01  3.52977604e-01
 -2.70589650e-01 -2.74643660e-01  9.76396143e-01  2.75295191e-02
 -1.72976360e-01 -3.24391186e-01  2.51091659e-01 -4.87938821e-01
 -2.64988899e-01 -1.97252899e-01  3.39790583e-01  1.06289601e+00
  7.10411072e-01  2.94107050e-01  1.32248491e-01 -8.32396448e-01
 -4.77076501e-01  9.51810628e-02  4.36197579e-01  4.02815491e-01
  2.80809462e-01 -2.63939917e-01  1.43596768e-01 -9.29925919e-01
  5.53071082e-01 -7.42091537e-01  5.26987351e-02  6.54800951e-01
  1.50363967e-01 -5.61388552e-01  1.69425309e-02  3.33137304e-01
 -1.05383313e+00 -8.75933468e-01  7.03832686e-01 -3.03714164e-02
  4.67144072e-01 -1.02644241e+00 -8.53308216e-02 -4.02355582e-01
  1.45488179e+00 -5.06986566e-02  3.23796958e-01 -3.92366111e-01
  6.37176096e-01  8.17435741e-01  7.92666733e-01 -4.28035446e-02
 -7.80105889e-01 -4.05690461e-01  6.15278542e-01 -5.23084283e-01
  1.63904592e-01 -5.43828368e-01  2.76540965e-01  7.24818766e-01
 -1.38656152e-02 -1.54290044e+00  5.89252532e-01  4.13415641e-01
  5.18130362e-02 -9.87655520e-02  4.82763559e-01 -8.47949207e-01
 -6.55694306e-01 -4.74998891e-01 -3.77971619e-01  2.01660290e-01
 -5.95442772e-01 -1.19410479e+00 -1.59502745e-01 -5.20548113e-02
 -2.21482903e-01  7.67169952e-01 -1.33745223e-02  3.95821542e-01
 -3.59742977e-02  9.76920426e-01  2.76537389e-01  1.86924899e+00
 -1.59637004e-01  1.57518223e-01  5.76984465e-01 -5.91236234e-01
 -6.67543113e-01  2.14618444e-01 -7.86880314e-01  4.59368080e-01
 -2.42100850e-01  3.72235328e-01  7.17906117e-01 -3.28328788e-01
 -8.23725045e-01 -7.59663939e-01 -5.52301168e-01 -5.82919978e-02
 -4.37329471e-01  5.81061900e-01  6.81886971e-01  2.07918197e-01
 -1.15603395e-01 -1.16515197e-01 -3.26178409e-02  2.22236469e-01
 -4.01333719e-01 -6.48582727e-02 -2.06386805e-01  2.96328396e-01
 -8.46319854e-01 -3.18517387e-01 -3.95449921e-02  3.39623064e-01
 -6.73294067e-01  2.80703098e-01 -2.19028145e-02 -1.92899272e-01
 -4.63035345e-01 -8.65612090e-01 -4.55583036e-01 -1.96341142e-01
 -6.63000569e-02  3.52349520e-01  2.36888885e-01 -1.11356175e+00
  1.07802711e-01 -1.45726845e-01  3.26886207e-01 -6.03936970e-01
 -3.76280725e-01 -2.30127096e-01  3.60069349e-02 -6.27485430e-03]

pip install scipy

Requirement already satisfied: scipy in /home/ubuntu/anaconda3/lib/python3.7/site-packages (1.4.1)
Requirement already satisfied: numpy>=1.13.3 in /home/ubuntu/anaconda3/lib/python3.7/site-packages (from scipy) (1.19.0)
Note: you may need to restart the kernel to use updated packages.

import scipy

#@title Sematic Search Form

# code adapted from https://github.com/UKPLab/sentence-transformers/blob/master/examples/application_semantic_search.py

#query = 'Nobody has sane thoughts' #@param {type: 'string'}
query = sentences[0]

queries = [query]
query_embeddings = model.encode(queries)

# Find the closest 3 sentences of the corpus for each query sentence based on cosine similarity
number_top_matches = 3 #@param {type: "number"}

print("Semantic Search Results")

for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], sentence_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for idx, distance in results[0:number_top_matches]:
        print(sentences[idx].strip(), "(Cosine Score: %.4f)" % (1-distance))

Semantic Search Results


======================


Query: I am Saeedeh

Top 5 most similar sentences in corpus:
I am Saeedeh (Cosine Score: 1.0000)
My name is Saeedeh (Cosine Score: 0.9577)

 

 

